In [27]:
# import random as rnd
# import secrets as scr
# from itertools import islice

from datetime import datetime
from random import SystemRandom

rnd = SystemRandom()


class Lotteria:

    def __init__(self,
                 max_numbers=90,
                 max_extra=90,
                 len_numbers=6,
                 len_extra=1):
        self.max_numbers = max_numbers
        self.max_extra = max_extra
        self.len_numbers = len_numbers
        self.len_extra = len_extra

    @property
    def combo(self):
        return self._combo

    @property
    def extra(self):
        return self._extra

    @property
    def backend(self):
        return self._backend.__name__

    @property
    def many(self):
        return self._many

    @staticmethod
    def abacus(max):
        for _ in iter(int, 1):
            yield rnd.randint(1, max)

    @staticmethod
    def choice(len, max):
        all_numbers = list(range(1, max+1))

        def extraction():
            sample = rnd.choice(all_numbers)
            all_numbers.remove(sample)

            return sample

        return frozenset((extraction() for _ in range(len)))

    @staticmethod
    def sample(len, max):
        all_numbers = tuple(range(1, max))

        return frozenset(
            rnd.sample(all_numbers, k=len))

    def randint(self, _len, max):
        combo = set()

        while len(combo) < _len:
            combo.add(next(self.abacus(max)))

        return frozenset(combo)

    def manySamples(self):
        '''
        To add further randomness, this method simulates several extractions 
        and pick one of them, hopefully the winning one :D
        '''
        size = self._many or 1

        def sampler():
            stop = rnd.randint(1, size)
            samples = (self.extract() for _ in range(stop))

            for s in samples:
                sample = s

            return sample  # type: ignore
        return sampler

    def extract(self):
        combo = self._backend(self.len_numbers, self.max_numbers)
        extra = self._backend(self.len_extra, self.max_extra) or None

        return combo, extra

    def __call__(self, backend='choice', many=None):
        self._many = many
        self._backend = eval(backend,
                             {'__builtins__': {}},
                             {'choice': self.choice,
                              'randint': self.randint,
                              'sample': self.sample})

        self._combo, self._extra = self.manySamples()()

        return self

    @property
    def draw(self):
        now = datetime.now().strftime("%d/%m/%Y %H:%M")

        print('Estrazione del:', now, '\nNumeri Estratti:',
              *sorted(self._combo))  # type: ignore
        if self._extra is not None:
            print('Superstar:', *sorted(self._extra))  # type: ignore

In [28]:
superenalotto = Lotteria(max_numbers=90, max_extra=90, len_numbers=6, len_extra=1)
superenalotto(backend='choice', many=500_000).draw

Estrazione del: 24/06/2023 09:14 
Numeri Estratti: 5 8 11 20 44 83
Superstar: 67


In [11]:
eurojack = Lotteria(max_numbers=50, max_extra=12, len_numbers=5, len_extra=2)
eurojack(backend='sample', many=500_000).draw

Estrazione del: 24/06/2023 08:53 
Numeri Estratti: 1 4 20 28 40
Superstar: 2 3


In [13]:
winForLife = Lotteria(20, 20, 10, 1)
winForLife(backend='sample', many=100_000).draw

Estrazione del: 24/06/2023 08:53 
Numeri Estratti: 1 4 8 10 12 14 15 16 17 19
Superstar: 4


In [80]:
superenalotto.len_extra = 1

In [81]:
superenalotto(backend='randint').draw

1
Estrazione del: 23/06/2023 20:58 
Numeri Estratti: 30 36 40 53 71 89
Superstar: 82


In [82]:
superenalotto.backend

'randint'

In [83]:
superenalotto.__dict__

{'max_numbers': 90,
 'max_extra': 90,
 'len_numbers': 6,
 'len_extra': 1,
 '_many': None,
 '_backend': <bound method Lotteria.randint of <__main__.Lotteria object at 0x000001648284D150>>,
 '_combo': frozenset({30, 36, 40, 53, 71, 89}),
 '_extra': frozenset({82})}

In [73]:
# trying to set backend via attribute: fails as expected
winForLife.backend = 'int'

AttributeError: property 'backend' of 'Lotteria' object has no setter

In [ ]:
# for b in ['choice', 'int', 'sample']:
#    print('######')
#    eurojack(backend=b, many=0)
#    print('backend:', eurojack.backend)
#
# print('\n################# many ####################\n')
#
# for b in ['choice', 'int', 'sample']:
#    print('######')
#    eurojack(backend=b)
#    print('backend:', eurojack.backend)

In [74]:
win = Lotteria(20, 20, 10, 1)

w_1 = frozenset({2, 3, 5, 7, 9, 11, 12, 14, 16, 18})
w_2 = frozenset({7})

In [75]:
i = 1
win()
x_1, x_2 = win.combo, win.extra
while ((w_1 != x_1) and (not w_1.isdisjoint(x_1))) or (w_2 != x_2):
    win()
    x_1, x_2 = win.combo, win.extra
    i += 1
    if i % 1_000_000 == 0:
        print(i)
print(i, '>>>', x_1, x_2)

180822 >>> frozenset({2, 3, 5, 7, 9, 11, 12, 14, 16, 18}) frozenset({7})


In [ ]:
#questo generatore infinito è stoppabile modificando alla bisogna il valore di clean a True in un ciclo FOR
clean = False
iterator = iter(lambda: clean, not clean)

#questo iterator itera all'infinito perchè int() ritorna 0
infinity = iter(int, 1)
